# Documentation of the test cases in test_aabb.cpp
### Troubleshooting
In case this notebook isn't working, make sure the matplotlib widgets are installed and enabled:
https://stackoverflow.com/a/55848505

In [2]:
%matplotlib widget

import numpy as np
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [122]:
def get_bb(bmin, bmax):   
    # this function calculates a meshgrid bounding box for easy use in matplotlib
    bmin = bmin.astype(np.float)
    bmax = bmax.astype(np.float)
    phi = np.arange(1,10,2)*np.pi/4
    Phi, Theta = np.meshgrid(phi, phi)
    
    # enhance visuability for points and lines
    for i in range(3): 
        if bmax[i] == bmin[i]:            
            bmax[i] = bmax[i] + 0.01

    x_scale = abs(bmax[0]-bmin[0])
    y_scale = abs(bmax[1]-bmin[1])
    z_scale = abs(bmax[2]-bmin[2])

    x = (np.cos(Phi)*np.sin(Theta)+0.5) * x_scale + bmin[0]
    y = (np.sin(Phi)*np.sin(Theta)+0.5) * y_scale + bmin[1]
    z = (np.cos(Theta)/np.sqrt(2)+0.5)  * z_scale + bmin[2]
    
    return x,y,z

def calc_ray(ray_dist, inter_min, inter_max):
    # this function calculates the ray parameters that are later used in visualization and testing
    ray_dir = (inter_max-inter_min)
    
    if (inter_max == inter_min).all():
        ray_dir = inter_min
        ray_org = np.array([0,0,0])

        t_min = np.linalg.norm(inter_min - ray_org)
        t_max = np.linalg.norm(inter_max - ray_org)
        #ray_dir = ray_dir / np.linalg.norm(ray_dir)
        ray_inv_dir = 1/ray_dir
        inf_index = np.where(ray_dir == (np.inf or -np.inf))
        for i in inf_index:
            ray_dir[i] = 1000 * np.sign(ray_dir[i])

    else:
        if np.linalg.norm(ray_dir) == np.inf:
            ray_org = np.array([0,0,0])
            t_min = np.linalg.norm(inter_min - ray_org)
            t_max = np.linalg.norm(inter_max - ray_org)

            for i in range(3):
                if inter_min[i] == (np.inf or -np.inf):
                    inter_min[i] = 1000 * np.sign(inter_min[i])
                if inter_max[i] == (np.inf or -np.inf):
                    inter_max[i] = 1000 * np.sign(inter_max[i])            
            ray_dir = (inter_max-inter_min)

            ray_dir = ray_dir / np.linalg.norm(ray_dir)
            ray_inv_dir = 1/ray_dir

        else:
            ray_dir = ray_dir / np.linalg.norm(ray_dir)
            ray_inv_dir = 1/ray_dir

            ray_org = inter_min - (ray_dir * ray_dist)     

            t_min = np.linalg.norm(inter_min - ray_org)
            t_max = np.linalg.norm(inter_max - ray_org)
    
    ray_dir_sign = np.sign(ray_dir)
    for i in range(len(ray_dir_sign)):
        if ray_dir_sign[i] >= 0:
            ray_dir_sign[i] = 0
        else:
            ray_dir_sign[i] = 1
    ray_dir_sign = ray_dir_sign.astype(np.int)
    
    return ray_org, ray_dir, ray_inv_dir, ray_dir_sign, t_min, t_max

def plot(bmin, bmax, inter_min, inter_max, ray_dist):
    # this function tries to visualize the testcase by plotting the bounding box, the ray and intersections
    # it also returns the values needed for testing    
    orig_vars = copy.deepcopy([bmin, bmax, inter_min, inter_max, ray_dist])
    vars_2 = copy.deepcopy([bmin, bmax, inter_min, inter_max, ray_dist])

    for v in range(len(vars_2)):
            try:
                for i in range(len(vars_2[v])):
                    if vars_2[v][i] == np.inf:
                        vars_2[v][i] = 1000
                    elif vars_2[v][i] == -np.inf:
                        vars_2[v][i] = -1000                 
            except:
                if vars_2[v] == np.inf:
                    vars_2[v] = 1000
                elif vars_2[v] == -np.inf:
                    vars_2[v] = -1000

    [bmin, bmax, inter_min, inter_max, ray_dist] = vars_2
        
    bmin = np.array(bmin)
    bmax = np.array(bmax)
    inter_min = np.array(inter_min)
    inter_max = np.array(inter_max)
    
    ray_org, ray_dir, ray_inv_dir, ray_dir_sign, t_min, t_max = calc_ray(np.array(orig_vars[4]), np.array(orig_vars[2]), np.array(orig_vars[3]))
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    x, y, z = get_bb(bmin, bmax)

    if t_max != (0 or np.inf or -np.inf):
        line = np.linspace(ray_org, ray_org + ray_dir * t_max, 100) 
    elif t_max == 0:
        line = np.linspace(ray_org, ray_org + ray_dir * 1, 100) 
    elif t_max == (np.inf or -np.inf):
        line = np.linspace(ray_org, ray_org + ray_dir, 100) 
    
    # initialize with viewing angle towards yz-plane
    ax.view_init(10, 1)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    
    # plot bounding box
    ax.plot_surface(x, y, z, alpha=0.2)
    
    # plot ray origin
    ax.scatter(ray_org[0], ray_org[1], ray_org[2])
    
    # plot intersection points
    ax.scatter(inter_min[0], inter_min[1], inter_min[2])
    ax.scatter(inter_max[0], inter_max[1], inter_max[2])    
    
    # plot ray
    ax.plot(line[:,0], line[:,1], line[:,2])
    
    plt.show()
    
    return ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max

def convert_testcase(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign, dtype = 'double', require = True, name = 'EXAMPLE'):
    # this function takes all relevant values and generates c++ code of test cases (subcases) to be used in a doctest environment
    
    template = '''
    SUBCASE("{20}")
    {{
    // highest numbers
    {0} pos_inf = std::numeric_limits<T>::max();
    {0} neg_inf = -1 * std::numeric_limits<T>::max();

    // return variables    
    {0} tmin;
    {0} tmax;    
    
    // correct values
    {0} tmin_cor = {18};
    {0} tmax_cor = {19};
        
    // parameters
    {0} min_t = {1};
    {0} max_t = {2};
    Vec3r<{0}> bmin{{ {3}, {4}, {5} }};
    Vec3r<{0}> bmax{{ {6}, {7}, {8} }};
    Vec3r<{0}> ray_org{{ {9}, {10}, {11} }};
    Vec3r<{0}> ray_inv_dir{{ {12}, {13}, {14} }};
    Vec3ui ray_dir_sign{{ {15}, {16}, {17} }};
    '''

    if require:
        template = template + '''
    CHECK(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    CHECK(tmin == Approx(tmin_cor));
    CHECK(tmax == Approx(tmax_cor));
    }}
    '''
    else:
        template = template + '''
    CHECK_FALSE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    }}
    '''                

    # translate the variables to the right types/strings
    vars = [bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max, min_t, max_t]
    for v in range(len(vars)):
            if dtype == 'double':
                vars[v] = np.array(vars[v]).astype(np.float64)
            elif dtype == 'float':
                vars[v] = np.array(vars[v]).astype(np.float32)
            else:
                vars[v] = np.array(vars[v]).astype(np.float64)
                
            vars[v] = np.array(vars[v]).astype(np.str)

            try:
                for i in range(len(vars[v])):
                    if vars[v][i] == 'inf':
                        vars[v][i] = 'pos_inf'
                    elif vars[v][i] == '-inf':
                        vars[v][i] = 'neg_inf'                    
            except:
                if vars[v] == 'inf':
                    vars[v] = 'pos_inf'
                elif vars[v] == '-inf':
                    vars[v] = 'neg_inf'  

    [bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign, tmin, tmax, min_t, max_t] = vars
    
    # fill the template with the given variables
    template = template.format(dtype, 
                               min_t, max_t, 
                               bmin[0], bmin[1], bmin[2], 
                               bmax[0], bmax[1], bmax[2],
                               ray_org[0], ray_org[1], ray_org[2],
                               ray_inv_dir[0], ray_inv_dir[1], ray_inv_dir[2],
                               int(float(ray_dir_sign[0])), int(float(ray_dir_sign[1])), int(float(ray_dir_sign[2])),
                               tmin, tmax,
                               name
                              )
    
    # show the c++ code
    print(template)

## Testcases

In [123]:
## basic example, nothing special here
# parameters
min_t = 0
max_t = 3
bmin = [-1, 1, 1]
bmax = [1, 2, 2]
inter_min = [0, 1, 1.5]
inter_max = [0, 2, 1.3]
ray_dist = (max_t - min_t)/2

# calculate ray_org, ray_inv_dir, t_min, t_max and visualize the test case
ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max = plot(bmin, bmax, inter_min, inter_max, ray_dist)

# convert to c++ test code
convert_testcase(t_min, t_max,
                 min_t, max_t,
                 bmin, bmax, 
                 ray_org, 
                 ray_inv_dir, ray_dir_sign, 
                 dtype = 'T', 
                 require = True,
                 name = 'basic_case'
                )

<ipython-input-122-317ca8b0a5f5>:57: RuntimeWarning: divide by zero encountered in true_divide
  ray_inv_dir = 1/ray_dir
<ipython-input-122-317ca8b0a5f5>:102: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


    SUBCASE("basic_case")
    {
    // highest numbers
    T pos_inf = std::numeric_limits<T>::max();
    T neg_inf = -1 * std::numeric_limits<T>::max();

    // return variables    
    T tmin;
    T tmax;    
    
    // correct values
    T tmin_cor = 1.5000000000000002;
    T tmax_cor = 2.519803902718557;
        
    // parameters
    T min_t = 0.0;
    T max_t = 3.0;
    Vec3r<T> bmin{ -1.0, 1.0, 1.0 };
    Vec3r<T> bmax{ 1.0, 2.0, 2.0 };
    Vec3r<T> ray_org{ 0.0, -0.4708710135363803, 1.794174202707276 };
    Vec3r<T> ray_inv_dir{ pos_inf, 1.019803902718557, -5.099019513592786 };
    Vec3ui ray_dir_sign{ 0, 0, 1 };
    
    REQUIRE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    REQUIRE(tmin == Approx(tmin_cor));
    REQUIRE(tmax == Approx(tmax_cor));
    }
    


In [124]:
## 0D bounding box (point), ray from origin
# parameters
min_t = 0
max_t = 3
bmin = [-1, 1, 1]
bmax = [-1, 1, 1]
inter_min = [-1, 1, 1]
inter_max = [-1, 1, 1]
ray_dist = (max_t - min_t)/2

# calculate ray_org, ray_inv_dir, t_min, t_max and visualize the test case
ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max = plot(bmin, bmax, inter_min, inter_max, ray_dist)

# convert to c++ test code
convert_testcase(t_min, t_max,
                 min_t, max_t,
                 bmin, bmax, 
                 ray_org, 
                 ray_inv_dir, ray_dir_sign, 
                 dtype = 'T', 
                 require = True,
                 name = 'BB_point'
                )

<ipython-input-122-317ca8b0a5f5>:102: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


    SUBCASE("BB_point")
    {
    // highest numbers
    T pos_inf = std::numeric_limits<T>::max();
    T neg_inf = -1 * std::numeric_limits<T>::max();

    // return variables    
    T tmin;
    T tmax;    
    
    // correct values
    T tmin_cor = 1.7320508075688772;
    T tmax_cor = 1.7320508075688772;
        
    // parameters
    T min_t = 0.0;
    T max_t = 3.0;
    Vec3r<T> bmin{ -1.0, 1.0, 1.0 };
    Vec3r<T> bmax{ -1.0, 1.0, 1.0 };
    Vec3r<T> ray_org{ 0.0, 0.0, 0.0 };
    Vec3r<T> ray_inv_dir{ -1.0, 1.0, 1.0 };
    Vec3ui ray_dir_sign{ 1, 0, 0 };
    
    REQUIRE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    REQUIRE(tmin == Approx(tmin_cor));
    REQUIRE(tmax == Approx(tmax_cor));
    }
    


In [129]:
np.sqrt(3)

1.7320508075688772

In [125]:
## 0D bounding box (point) at 'infinity', ray from origin
# parameters
min_t = 0
max_t = np.inf
bmin = [-1, np.inf, 1]
bmax = [-1, np.inf, 1]
inter_min = [-1, np.inf, 1]
inter_max = [-1, np.inf, 1]
ray_dist = 1000

# calculate ray_org, ray_inv_dir, t_min, t_max and visualize the test case
ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max = plot(bmin, bmax, inter_min, inter_max, ray_dist)

# convert to c++ test code
convert_testcase(t_min, t_max,
                 min_t, max_t,
                 bmin, bmax, 
                 ray_org, 
                 ray_inv_dir, ray_dir_sign, 
                 dtype = 'T', 
                 require = True,
                 name = 'BB_point_inf'
                )

<ipython-input-122-317ca8b0a5f5>:25: RuntimeWarning: invalid value encountered in subtract
  ray_dir = (inter_max-inter_min)
<ipython-input-122-317ca8b0a5f5>:102: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


    SUBCASE("BB_point_inf")
    {
    // highest numbers
    T pos_inf = std::numeric_limits<T>::max();
    T neg_inf = -1 * std::numeric_limits<T>::max();

    // return variables    
    T tmin;
    T tmax;    
    
    // correct values
    T tmin_cor = pos_inf;
    T tmax_cor = pos_inf;
        
    // parameters
    T min_t = 0.0;
    T max_t = pos_inf;
    Vec3r<T> bmin{ -1.0, pos_inf, 1.0 };
    Vec3r<T> bmax{ -1.0, pos_inf, 1.0 };
    Vec3r<T> ray_org{ 0.0, 0.0, 0.0 };
    Vec3r<T> ray_inv_dir{ -1.0, 0.0, 1.0 };
    Vec3ui ray_dir_sign{ 1, 0, 0 };
    
    REQUIRE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    REQUIRE(tmin == Approx(tmin_cor));
    REQUIRE(tmax == Approx(tmax_cor));
    }
    


In [126]:
## finite 1D bounding box (line), ray from origin
# parameters
min_t = 0
max_t = 3
bmin = [-1, 1, 1]
bmax = [-1, 5, 1]
inter_min = [-1, 2, 1]
inter_max = [-1, 2, 1]
ray_dist = (max_t - min_t)/2

# calculate ray_org, ray_inv_dir, t_min, t_max and visualize the test case
ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max = plot(bmin, bmax, inter_min, inter_max, ray_dist)

# convert to c++ test code
convert_testcase(t_min, t_max,
                 min_t, max_t,
                 bmin, bmax, 
                 ray_org, 
                 ray_inv_dir, ray_dir_sign, 
                 dtype = 'T', 
                 require = True,
                 name = 'BB_line'
                )

<ipython-input-122-317ca8b0a5f5>:102: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


    SUBCASE("BB_line")
    {
    // highest numbers
    T pos_inf = std::numeric_limits<T>::max();
    T neg_inf = -1 * std::numeric_limits<T>::max();

    // return variables    
    T tmin;
    T tmax;    
    
    // correct values
    T tmin_cor = 2.449489742783178;
    T tmax_cor = 2.449489742783178;
        
    // parameters
    T min_t = 0.0;
    T max_t = 3.0;
    Vec3r<T> bmin{ -1.0, 1.0, 1.0 };
    Vec3r<T> bmax{ -1.0, 5.0, 1.0 };
    Vec3r<T> ray_org{ 0.0, 0.0, 0.0 };
    Vec3r<T> ray_inv_dir{ -1.0, 0.5, 1.0 };
    Vec3ui ray_dir_sign{ 1, 0, 0 };
    
    REQUIRE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    REQUIRE(tmin == Approx(tmin_cor));
    REQUIRE(tmax == Approx(tmax_cor));
    }
    


In [127]:
## 'infinite' 1D bounding box (line), ray from origin
# parameters
min_t = 0
max_t = 3
bmin = [-1, 1, 1]
bmax = [-1, np.inf, 1]
inter_min = [-1, 2, 1]
inter_max = [-1, 2, 1]
ray_dist = (max_t - min_t)/2

# calculate ray_org, ray_inv_dir, t_min, t_max and visualize the test case
ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max = plot(bmin, bmax, inter_min, inter_max, ray_dist)

# convert to c++ test code
convert_testcase(t_min, t_max,
                 min_t, max_t,
                 bmin, bmax, 
                 ray_org, 
                 ray_inv_dir, ray_dir_sign, 
                 dtype = 'T', 
                 require = True,
                 name = 'BB_line_inf'
                )

<ipython-input-122-317ca8b0a5f5>:102: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


    SUBCASE("BB_line_inf")
    {
    // highest numbers
    T pos_inf = std::numeric_limits<T>::max();
    T neg_inf = -1 * std::numeric_limits<T>::max();

    // return variables    
    T tmin;
    T tmax;    
    
    // correct values
    T tmin_cor = 2.449489742783178;
    T tmax_cor = 2.449489742783178;
        
    // parameters
    T min_t = 0.0;
    T max_t = 3.0;
    Vec3r<T> bmin{ -1.0, 1.0, 1.0 };
    Vec3r<T> bmax{ -1.0, pos_inf, 1.0 };
    Vec3r<T> ray_org{ 0.0, 0.0, 0.0 };
    Vec3r<T> ray_inv_dir{ -1.0, 0.5, 1.0 };
    Vec3ui ray_dir_sign{ 1, 0, 0 };
    
    REQUIRE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    REQUIRE(tmin == Approx(tmin_cor));
    REQUIRE(tmax == Approx(tmax_cor));
    }
    


In [128]:
## 2D bounding box (plane), ray from origin
# parameters
min_t = 0
max_t = 3
bmin = [-1, 1, 1]
bmax = [-1, 5, 5]
inter_min = [-1, 2, 3]
inter_max = [-1, 2, 3]
ray_dist = (max_t - min_t)/2

# calculate ray_org, ray_inv_dir, t_min, t_max and visualize the test case
ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max = plot(bmin, bmax, inter_min, inter_max, ray_dist)

# convert to c++ test code
convert_testcase(t_min, t_max,
                 min_t, max_t,
                 bmin, bmax, 
                 ray_org, 
                 ray_inv_dir, ray_dir_sign, 
                 dtype = 'T', 
                 require = True,
                 name = 'BB_plane'
                )

<ipython-input-122-317ca8b0a5f5>:102: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


    SUBCASE("BB_plane")
    {
    // highest numbers
    T pos_inf = std::numeric_limits<T>::max();
    T neg_inf = -1 * std::numeric_limits<T>::max();

    // return variables    
    T tmin;
    T tmax;    
    
    // correct values
    T tmin_cor = 3.7416573867739413;
    T tmax_cor = 3.7416573867739413;
        
    // parameters
    T min_t = 0.0;
    T max_t = 3.0;
    Vec3r<T> bmin{ -1.0, 1.0, 1.0 };
    Vec3r<T> bmax{ -1.0, 5.0, 5.0 };
    Vec3r<T> ray_org{ 0.0, 0.0, 0.0 };
    Vec3r<T> ray_inv_dir{ -1.0, 0.5, 0.3333333333333333 };
    Vec3ui ray_dir_sign{ 1, 0, 0 };
    
    REQUIRE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    REQUIRE(tmin == Approx(tmin_cor));
    REQUIRE(tmax == Approx(tmax_cor));
    }
    


In [15]:
# parameters
min_t = 0
max_t = 5
bmin = [-1, 1, 1]
bmax = [2,2,2]
inter_min = [-1, 1, 1]
inter_max = [2,2,2]
ray_dist = (max_t - min_t)/2

# calculate ray_org, ray_inv_dir, t_min, t_max and visualize the test case
ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max = plot(bmin, bmax, inter_min, inter_max, ray_dist)

# convert to c++ test code
convert_testcase(t_min, t_max,
                 min_t, max_t,
                 bmin, bmax, 
                 ray_org, 
                 ray_inv_dir, ray_dir_sign, 
                 dtype = 'T', 
                 require = True,
                 name = 'BB_flat_infinte'
                )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


    SUBCASE("BB_flat_infinte")
    {
    // highest numbers
    T pos_inf = <T>::max();
    T neg_inf = -1 * <T>::max();

    // return variables    
    T tmin;
    T tmax;    
    
    // correct values
    T tmin_cor = 2.5;
    T tmax_cor = 5.8166247903554;
        
    // parameters
    T min_t = 0.0;
    T max_t = 5.0;
    Vec3r<T> bmin{ -1.0, 1.0, 1.0 };
    Vec3r<T> bmax{ 2.0, 2.0, 2.0 };
    Vec3r<T> ray_org{ -3.2613350843332274, 0.24622163855559087, 0.24622163855559087 };
    Vec3r<T> ray_inv_dir{ 1.1055415967851332, 3.3166247903554, 3.3166247903554 };
    Vec3ui ray_dir_sign{ 0, 0, 0 };
    
    REQUIRE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    REQUIRE(tmin == Approx(tmin_cor));
    REQUIRE(tmax == Approx(tmax_cor));
    }
    


In [74]:
# parameters
min_t = 0
max_t = np.inf
bmin = [-1, 1, 1]
bmax = [np.inf, np.inf, np.inf]
inter_min = [-1, 1, 1]
inter_max = [np.inf, np.inf, np.inf]
ray_dist = 500

# calculate ray_org, ray_inv_dir, t_min, t_max and visualize the test case
ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max = plot(bmin, bmax, inter_min, inter_max, ray_dist)
print(ray_org, ray_inv_dir, ray_dir_sign, t_min, t_max)
print(bmax)

# convert to c++ test code
convert_testcase(t_min, t_max,
                 min_t, max_t,
                 bmin, bmax, 
                 ray_org, 
                 ray_inv_dir, ray_dir_sign, 
                 dtype = 'T', 
                 require = True,
                 name = 'BB_flat_infinte'
                )

[inf, inf, inf]
[inf, inf, inf]


<ipython-input-66-a2c2d9e3d11b>:85: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[-290.06003456 -287.48249203 -287.48249203] [1.72974448 1.73320743 1.73320743] [0 0 0] 500.00000000000006 2231.4742273565607
[inf, inf, inf]

    SUBCASE("BB_flat_infinte")
    {
    // highest numbers
    T pos_inf = <T>::max();
    T neg_inf = -1 * <T>::max();

    // return variables    
    T tmin;
    T tmax;    
    
    // correct values
    T tmin_cor = 500.00000000000006;
    T tmax_cor = 2231.4742273565607;
        
    // parameters
    T min_t = 0.0;
    T max_t = pos_inf;
    Vec3r<T> bmin{ -1.0, 1.0, 1.0 };
    Vec3r<T> bmax{ pos_inf, pos_inf, pos_inf };
    Vec3r<T> ray_org{ -290.0600345603253, -287.4824920337312, -287.4824920337312 };
    Vec3r<T> ray_inv_dir{ 1.7297444828736874, 1.7332074347913522, 1.7332074347913522 };
    Vec3ui ray_dir_sign{ 0, 0, 0 };
    
    REQUIRE(IntersectRayAABB(tmin, tmax, min_t, max_t, bmin, bmax, ray_org, ray_inv_dir, ray_dir_sign));
    REQUIRE(tmin == Approx(tmin_cor));
    REQUIRE(tmax == Approx(tmax_cor));
    }
    


In [94]:
np.inf-0

inf